<a href="https://colab.research.google.com/github/RanulfoMNeto/Projeto-Final/blob/main/map_ploting_m3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***MACROENTREGA 3 - MAP PLOTING*** 

In [ ]:
import pandas as pd
import folium
from folium import plugins


> ### Lendo base de dados

In [ ]:
arquivo = input('Digite o nome do arquivo solução: ')
arquivo = arquivo + '.csv'

Digite o nome do arquivo solução: bar-n100-1(output)


In [ ]:
solution_df = pd.read_csv(arquivo)

In [ ]:
solution_df.head(20)

> ### **Extraindo a coordenada do Depósito** 

In [ ]:
latDep = solution_df.loc[0, 'lat']
lonDep = solution_df.loc[0, 'lon']
coordDep = [latDep,lonDep]

In [ ]:
coordDep

[41.3975366, 2.1235633]

> ### **Inserindo pontos de entrega, coleta e depósito no mapa** 

In [ ]:
def build_theMap(): 
  mapa = folium.Map(
                    location = coordDep,
                    tiles = 'cartodbpositron',
                    zoom_start = 13
                   )

  #adicionando layers de vizualização
  markers_ent = folium.FeatureGroup(name = 'Pontos de entrega')
  markers_col = folium.FeatureGroup(name = 'Pontos de coleta')
  #markers_rota = folium.FeatureGroup(name = 'Rota de um caminhão')

  folium.Marker(coordDep, 
             tooltip = '<i>Depósito</i>',
             icon = folium.Icon(color='green'),
             ).add_to(mapa)

  for id, lati, longi, deman in zip(solution_df.id, solution_df.lat, solution_df.lon, solution_df.dem):

    popupInfo = '<i>id do vértice: {} <br> <br> demanda do vértice: {}<i>'.format(id, deman)

    if deman > 0:
      markers_ent.add_child(folium.Marker([lati,longi], 
             tooltip = '<i>Coleta</i> <br> Clique para mais informações!',
             popup = folium.Popup(popupInfo,max_width=200,max_height=200),
             icon = folium.Icon(color='red')
             ))
    
    elif deman < 0:
      markers_col.add_child(folium.Marker([lati,longi],
             tooltip = '<i>Entrega</i> <br> Clique para mais informações!',
             popup = folium.Popup(popupInfo,max_width=200,max_height=200),
             icon = folium.Icon(color='blue')
             ))
    
  mapa.add_child(markers_ent)
  mapa.add_child(markers_col)
  mapa.add_child(folium.LayerControl())
  
  return mapa


In [ ]:
mapa = build_theMap()

In [ ]:
mapa

>> ### **Traçando a rota de todos os caminhões**

In [ ]:
solution_df = pd.read_csv(arquivo)
max = solution_df['m'].max()
m = 0

while m < max:
  j = 0
  i = 0

  coordenadas = []

  #preenchendo a lista de coordenadas
  for id, lat, lon in zip(solution_df.id,solution_df.lat,solution_df.lon):
    if i > 1 and id == 0:
      j+=1
    if j > 1 and id == 0:
      break
    coordenadas.append([lat,lon])

    i+=1

  #traçando a rota
  for it in range(len(coordenadas)-1):
    ponto_inicial = coordenadas[it]
    ponto_final = coordenadas[it+1]
    folium.PolyLine(locations = [ponto_inicial,ponto_final], color = 'purple').add_to(mapa)

  mapa.save('TodasAsRotas.html')

  if m > 1:
    df_remove.drop('m', axis = 1)

  df_remove = solution_df.loc[(solution_df['m'] == m)]
  solution_df = solution_df.drop(df_remove.index)
  m+=1

>>### **Mostrar uma rota de cada vez**

In [ ]:
solution_df = pd.read_csv(arquivo)
mapa = build_theMap()
max = solution_df['m'].max()
m = 0


flag = 1
while flag != 0 :
  j = 0
  i = 0

  coordenadas = []

  #preenchendo a lista de coordenadas
  for id, lat, lon in zip(solution_df.id,solution_df.lat,solution_df.lon):
    if i > 1 and id == 0:
      j+=1
    if j > 1 and id == 0:
      break
    coordenadas.append([lat,lon])

    i+=1

  #traçando a rota
  for it in range(len(coordenadas)-1):
    ponto_inicial = coordenadas[it]
    ponto_final = coordenadas[it+1]
    folium.PolyLine(locations = [ponto_inicial,ponto_final], color = 'purple').add_to(mapa)

  mapa.save('Rota1caminhao.html')

  if m > 1:
    df_remove.drop('m', axis = 1)

  df_remove = solution_df.loc[(solution_df['m'] == m)]
  solution_df = solution_df.drop(df_remove.index)
  m+=1

  
  print('Geramos um html com a rota do primeiro caminhão! Confira nos arquivos!')
  flag = int(input('Deseja ver a roteirização do próximo caminhão? \n (digite 1 para continuar e 0 para parar) \n'))
  mapa = build_theMap()

> ### **Gerando mapa de calor dos pontos de entrega**

In [ ]:
mapa = folium.Map(
                  location = coordDep,
                  tiles = 'cartodbpositron',
                  zoom_start = 12
                  )

folium.Marker(coordDep, 
             tooltip = '<i>Depósito</i>',
             icon = folium.Icon(color='green'),
             ).add_to(mapa)

coordEnt = []
for lati, longi, deman in zip(solution_df.lat, solution_df.lon, solution_df.dem):
  if deman < 0:
    coordEnt.append([lati,longi])
    folium.Marker([lati,longi],
             tooltip = '<i>Entrega</i>',
             icon = folium.Icon(color='blue')
             )

In [ ]:
mapa.add_child(plugins.HeatMap(coordEnt))